In [1]:
import pandas as pd
from datetime import datetime
import sys
from IPython.display import clear_output

/Users/valentin/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:1035: UserWarning: Duplicate key in file "/Users/valentin/anaconda/lib/python2.7/site-packages/matplotlib/mpl-data/matplotlibrc", line #513
  (fname, cnt))


In [2]:
date = []
chain_store = []
store_no = []
store_name = []
postal_no = []
EAN = []
Produkttekst = []
oms = []
volume = []
avance = []
errors = []
with open('RU udtrëk af salg af õl for 2015  - 20161026.csv') as f:
    lines = f.readlines()
    print(str(len(lines)) + " lines")
    k = len(lines)/100
    j = 0
    for l in lines:
        #clear_output()
        #print j
        #print len(splits)
        l = l.replace('4,5%' , '4.5%')
        l = l.replace('FAKTA,JENS BAGGESENSVEJ' , 'FAKTA - JENS BAGGESENSVEJ')
        splits = l.split(',')
        if len(splits) == 10:
            i = 0
        else:
            i = 1
        date.append(  datetime.strptime(splits[0].replace('\xef\xbb\xbf',''), '%Y-%m-%d') )
        chain_store.append(splits[1])
        store_no.append(int(splits[2]))
        store_name.append(splits[3])
        postal_no.append(int(splits[4+i]))
        EAN.append(splits[5+i])
        Produkttekst.append(splits[6+i])
        oms.append(float(splits[7+i]))
        volume.append(int(float(splits[8+i])))
        avance.append(int(float(splits[9+i][:-4])))

        j += 1
        if j % k == 0:
            clear_output()
            print str(int(float(j)/float(len(lines))*100.0)) + '%'

df = pd.DataFrame( {'date': date , 'chain_store' : chain_store, 'store_no': store_no, 'store_name':store_name,'postal_no':postal_no,
                   'EAN':EAN,'Produkttekst': Produkttekst, 'oms':oms, 'volume':volume, 'avance':avance})
print(len(df))

99%
6094460


In [8]:
df[:10]

,EAN,Produkttekst,avance,chain_store,date,oms,postal_no,store_name,store_no,volume,station
0,5741000116975,Royal B X-mas 6x33ds,1,DagliBrugsen,2015-01-01,31.00,2300,COOP DAGLIBR BRYDES ALLE,1043,1,618800
1,5741000131084,Royal Export 6x33 ds,-13,DagliBrugsen,2015-01-01,100.00,2300,COOP DAGLIBR BRYDES ALLE,1043,4,618800
2,5741000140284,ROYAL PLATINUM 6-PAK,8,DagliBrugsen,2015-01-01,39.95,2300,COOP DAGLIBR BRYDES ALLE,1043,1,618800
3,5741000131084,Royal Export 6x33 ds,17,DagliBrugsen,2015-01-01,79.90,2400,COOP DAGLIBR BISPEPARKEN,1053,2,618800
4,5741000132463,Royal Exp. kuffert24,-19,DagliBrugsen,2015-01-01,299.85,2650,COOP LOKALBR FJELDSTEDVEJ,1082,3,618800
5,5741000131084,Royal Export 6x33 ds,9,DagliBrugsen,2015-01-01,39.95,4000,COOP DAGLIBR GEVNINGE,1099,1,617400
6,5741000116494,Royal Class. 6x33 ds,8,DagliBrugsen,2015-01-01,38.95,4000,DAGLIBRUGSEN GUNDSØMAGLE,1104,1,617400
7,5741000125021,Royal Stout 33 cl ds,6,DagliBrugsen,2015-01-01,21.50,4000,DAGLIBRUGSEN GUNDSØMAGLE,1104,2,617400
8,5741000132456,Royal Class. 24ds,-4,DagliBrugsen,2015-01-01,99.95,4000,DAGLIBRUGSEN GUNDSØMAGLE,1104,1,617400
9,5741000132463,Royal Exp. kuffert24,-6,DagliBrugsen,2015-01-01,99.95,4000,DAGLIBRUGSEN GUNDSØMAGLE,1104,1,617400


## add weather data 

In [3]:
def zipToRegion(zip):
    
    # 1-2 Copenhagen
    if (zip >= 1000 and zip <= 2999):
        return 618800
        
    # 3 Sealand
    if (zip >= 3000 and zip <= 3699) or (zip >= 4000 and zip <= 4999):
        return 617400
        
    # 5 Fyn and the islands
    if (zip >= 5000 and zip <= 5999):
        return 612600
        
    # 6 Southern/Western Jutland
    if (zip >= 6000 and zip <= 6999):
        return 6116
    
    # skip th
    if (zip >= 7000 and zip <= 7999):
        return 0
        
    # 7 Eastern Jutland
    if (zip >= 8000 and zip <= 8999):
        return 607200
    
    # 9 Northern Jutland
    if (zip >= 9000 and zip <= 9999):
        return 603100

In [4]:
df['station'] = df['postal_no'].map(zipToRegion)
df[:3]

,EAN,Produkttekst,avance,chain_store,date,oms,postal_no,store_name,store_no,volume,station
0,5741000116975,Royal B X-mas 6x33ds,1,DagliBrugsen,2015-01-01,31.00,2300,COOP DAGLIBR BRYDES ALLE,1043,1,618800
1,5741000131084,Royal Export 6x33 ds,-13,DagliBrugsen,2015-01-01,100.00,2300,COOP DAGLIBR BRYDES ALLE,1043,4,618800
2,5741000140284,ROYAL PLATINUM 6-PAK,8,DagliBrugsen,2015-01-01,39.95,2300,COOP DAGLIBR BRYDES ALLE,1043,1,618800


## load weather data

In [24]:
cols = ["station", "year", "month", "day", 
        "temp", "wind", "sun", "precipitations"]

df_weather = pd.read_csv("data/2015_weather_new.csv", sep=";", 
                         skiprows=1, names=cols)

df_weather['date'] = df_weather.year*10000 + df_weather.month*100 + df_weather.day
df_weather['date'] = df_weather['date'].map( lambda x: datetime.strptime( str(x) , '%Y%m%d'))
df_weather = df_weather.drop( ['year','month','day'] , 1)
df_weather[:10] 

,station,temp,wind,sun,precipitations,date
0,618800,5.5,6.1,0.0,1.9,2015-01-01
1,618800,5.9,9.3,2.2,1.3,2015-01-02
2,618800,3.9,6.5,3.3,1.6,2015-01-03
3,618800,3.4,4.9,4.1,0.0,2015-01-04
4,618800,1.8,1.2,0.0,1.0,2015-01-05
5,618800,2.1,3.6,0.0,0.0,2015-01-06
6,618800,3.3,3.8,5.4,1.1,2015-01-07
7,618800,5.1,4.8,0.0,2.8,2015-01-08
8,618800,5.2,7.7,0.8,7.5,2015-01-09
9,618800,5.6,8.1,0.2,10.0,2015-01-10


In [29]:
merged = pd.merge(df,df_weather, on=['date','station'])

In [30]:
merged

,EAN,Produkttekst,avance,chain_store,date,oms,postal_no,store_name,store_no,volume,station,temp,wind,sun,precipitations
0,5741000116975,Royal B X-mas 6x33ds,1,DagliBrugsen,2015-01-01,31.00,2300,COOP DAGLIBR BRYDES ALLE,1043,1,618800,5.5,6.1,0,1.9
1,5741000131084,Royal Export 6x33 ds,-13,DagliBrugsen,2015-01-01,100.00,2300,COOP DAGLIBR BRYDES ALLE,1043,4,618800,5.5,6.1,0,1.9
2,5741000140284,ROYAL PLATINUM 6-PAK,8,DagliBrugsen,2015-01-01,39.95,2300,COOP DAGLIBR BRYDES ALLE,1043,1,618800,5.5,6.1,0,1.9
3,5741000131084,Royal Export 6x33 ds,17,DagliBrugsen,2015-01-01,79.90,2400,COOP DAGLIBR BISPEPARKEN,1053,2,618800,5.5,6.1,0,1.9
4,5741000132463,Royal Exp. kuffert24,-19,DagliBrugsen,2015-01-01,299.85,2650,COOP LOKALBR FJELDSTEDVEJ,1082,3,618800,5.5,6.1,0,1.9
5,5741000116494,Royal Class. 6x33 ds,10,DagliBrugsen,2015-01-01,40.95,2500,COOP LOKALBR HERMAN BANGS PLAD,1134,1,618800,5.5,6.1,0,1.9
6,5741000117958,Royal Plsnr 6x33 ds,10,DagliBrugsen,2015-01-01,40.95,2500,COOP LOKALBR HERMAN BANGS PLAD,1134,1,618800,5.5,6.1,0,1.9
7,5741000131077,Royal Export ds,7,DagliBrugsen,2015-01-01,24.00,2500,COOP LOKALBR HERMAN BANGS PLAD,1134,2,618800,5.5,6.1,0,1.9
8,5741000132463,Royal Exp. kuffert24,-13,DagliBrugsen,2015-01-01,199.90,2610,COOP DAGLIBRUGSEN ISLEV TORV,1194,2,618800,5.5,6.1,0,1.9
9,5741000135013,HEINEKEN KUFFERT,-13,DagliBrugsen,2015-01-01,199.90,2610,COOP DAGLIBRUGSEN ISLEV TORV,1194,2,618800,5.5,6.1,0,1.9


### save to file

In [31]:
merged.to_csv('cleaned_sales+weather_data.csv')